<a href="https://colab.research.google.com/github/Disha-Sikka/ResNet18_from_scratch/blob/main/resnet_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import torch
import torch.nn as nn #for defining layers
from torchvision import datasets, transforms #for importing datasets and applying transformations
import torch.nn.functional as F
from torch.utils.data import DataLoader, Subset # for importing data in batches
import matplotlib.pyplot as plt # for plating data on graphs
import numpy as np #to do array transformations


In [ ]:
device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class BasicBlock(nn.Module): # nn.Module is the base class for all nerual networks
  def __init__(self, in_channels, out_channels, stride=1, downSample=None): # in_channels and out_channels for the depth of input and output image, stride for the no. of times a filter move over the input
    super(BasicBlock, self).__init__()
    self.conv1= nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
    self.bn1= nn.BatchNorm2d(out_channels)
    self.relu= nn.ReLU(inplace=True)
    self.conv2= nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
    self.bn2= nn.BatchNorm2d(out_channels)
    self.downSample= downSample

  def forward(self, x):
    identity= x
    if self.downSample:
      identity=self.downSample(x)

    out=self.conv1(x)
    out=self.bn1(out)
    out=self.relu(out)
    out=self.conv2(out)
    out=self.bn2(out)
    out+=identity
    out= self.relu(out)
    return out

In [8]:
class ResNet(nn.Module):
  def __init__(self, block, layers, num_classes=1000):
    super(ResNet18, self).__init__()
    self.in_channels=64

    self.conv1= nn.Conv2d(3, 64, kernel_size=3, padding=1, stride=1, bias=False)
    self.bn1= nn.BatchNorm2d(64)
    self.relu= nn.ReLU(True)

    self.layer1= self.make_layer(block, 64, layers[0], stride=1)
    self.layer2= self.make_layer(block, 64, layers[1], stride=2)
    self.layer3= self.make_layer(block, 64, layers[2], stride=2)
    self.layer4= self.make_layer(block, 64, layers[3], stride=2)

    self.avgpool= nn.AdaptiveAvgPool2d((1,1))
    self.fc= nn.Linear(512, num_classes)

    def make_layer(self,bloack, out_channels, blocks, stride):
      strides= [stride] + [1] * (blocks-1)
      layers=[]

      for s in strides:
        downSample= None

        if s!=1 or self.in_channels != out_channels:
          downSample= nn.Sequential(nn.Conv2d(self.in_channels, out_channels, kernel_size=3, padding=1, stride=1, bias= False)
          , self.BatchNorm2d(out_channels))

        layers.append(block(self.in_channels, out_channels, s, downSample))
        self.in_channels= out_channels

      return nn.Sequential(*layers)

    def forward(self, x):
      x= self.conv1(x)
      x= self.bn1(x)
      x= self.relu(x)

      x= self.layer1(x)
      x= self.layer2(x)
      x= self.layer3(x)
      x= self.layer4(x)

      x=self.avgpool(x)
      x=torch.flatten(x,1)
      x.self.fc(x)

      return x



In [7]:
def ResNet18():
  return (ResNet(BasicBlock, [2,2,2,2]))

In [9]:
from torchvision.models import resnet18

In [14]:
model= resnet18(pretrained=True)
model.fc =nn.Linear(512,10)
model=model.to(device)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [25]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_subset = Subset(train_dataset, range(0, 256))  # first 256 images
train_loader = DataLoader(train_subset, batch_size=8, shuffle=True)

100%|██████████| 170M/170M [00:12<00:00, 13.6MB/s]
